In [1]:
import requests
import pandas as pd

beefi_api_apy = 'https://api.beefy.finance/apy'
beefi_api_tvl = 'https://api.beefy.finance/tvl'
yearn_api = 'https://api.yexporter.io/v1/chains/10/vaults/all'

In [2]:
response_apy = requests.get(beefi_api_apy)
data_apy = response_apy.json()
filtered_data_apy = {k: v for k, v in data_apy.items() if 'velodrome' in k}
df_apy = pd.DataFrame(list(filtered_data_apy.items()), columns=['Pair', 'Beefy APY'])
response_tvl = requests.get(beefi_api_tvl)
data_tvl = response_tvl.json()

filtered_data_tvl = {}
for _, nested_dict in data_tvl.items():
    for k, v in nested_dict.items():
        if 'velodrome-' in k:
            filtered_data_tvl[k] = v

df_tvl = pd.DataFrame(list(filtered_data_tvl.items()), columns=['Pair', 'Beefy TVL'])
merged_beefy_df = pd.merge(df_apy, df_tvl, on='Pair', how='inner')
merged_beefy_df['Pair'] = merged_beefy_df['Pair'].str.replace('velodrome-v2-', '', regex=False).str.upper().str.replace('/', '-')
merged_beefy_df['Pair'] = merged_beefy_df['Pair'].str.replace('VELODROME', 'V1', regex=False)

In [3]:
response = requests.get(yearn_api)
data = response.json()
filtered_data = [vault for vault in data if any("StrategyVelodrome" in strategy['name'] or "yvVelo" in strategy['name'] for strategy in vault['strategies'])]

extracted_data = []
for vault in filtered_data:
    for strategy in vault['strategies']:
        pair_name = strategy['name']
        
        # Strip undesired parts and make replacements
        pair_name = pair_name.replace("StrategyVelodromeFactory-vAMMV2-", "").strip()
        pair_name = pair_name.replace("StrategyVelodromeFactory-sAMMV2-", "").strip()
        pair_name = pair_name.replace("StrategyVelodromeFactory", "V1-").strip()
        pair_name = pair_name.replace("StrategyVelodrome", "V1-").strip()
        pair_name = pair_name.replace("Clonable", "").strip()
        pair_name = pair_name.replace("USDCDOLA", "USDC-DOLA").strip()
        
        # Capitalize and replace "/"
        pair_name = pair_name.upper().replace("/", "-")

        extracted_data.append({
            'Pair': pair_name, 
            'Yearn APY': vault['apy']['net_apy'], 
            'Yearn TVL': vault['tvl']['tvl'],
            'Yearn Address': vault['address']
        })

yearn_df = pd.DataFrame(extracted_data)

In [4]:
# Merge the two DataFrames on the 'Pair' column
merge_df = pd.merge(merged_beefy_df, yearn_df, on='Pair', how='left')
print(merge_df)
# Export the final_df DataFrame to a CSV file
merge_df.to_csv('partial.csv', index=False)

              Pair Beefy APY   Beefy TVL  Yearn APY      Yearn TVL  \
0     V1-FRAX-USD+  2.731887        0.44        NaN            NaN   
1    V1-FRAX-ALUSD  0.000786      901.04        NaN            NaN   
2     V1-USD+-DAI+   0.00765    20116.04        NaN            NaN   
3      V1-LUSD-DAI  0.001845     1114.10        NaN            NaN   
4     V1-USDT-LUSD  0.000619       30.99        NaN            NaN   
..             ...       ...         ...        ...            ...   
146     USDC-WUSDR  0.645378  1130197.65   0.642253  144909.304815   
147       WETH-HOP  0.289775     2249.18        NaN            NaN   
148       JRT-WETH   0.16414       95.58        NaN            NaN   
149  WSTETH-PENDLE  0.025771    19641.44        NaN            NaN   
150       OVN-USD+  1.983483   480450.69        NaN            NaN   

                                  Yearn Address  
0                                           NaN  
1                                           NaN  
2        

In [5]:
# From CMO tx
# Careful, many pairs names are inverted

pools = {
  'ALETH-WETH': 2000,  
  'USDC-MAI': 1400,
  'ERN-DOLA': 1400, # not here???
  'WETH-LUSD': 1000,
  'USDC-VELO': 1000,
  'USDC-SNX': 1000,
  'USDC-ALUSD': 750,
  'USDC-AGEUR': 500, 
  'WETH-TBTC': 500,
  'USDC-MTA': 500,
  'STERN-ERN': 500, # not here???
  'STG-USDC': 500,
  'FRAX-ALUSD': 300,
  'WETH-OP': 300,
  'WSTETH-LDO': 300,
  'WBTC-TBTC': 300,
  'OP-USDC': 300,
  'USDC-DOLA': 300,
  'EXA-WETH': 300, 
  'FRAX-DOLA': 200,
  'ALUSD-MAI': 150,
  'OP-VELO': 150,
  'ALETH-FRXETH': 150,
  'IB-WETH': 150, # not here???
  'DOLA-MAI': 150,
  'ERN-LUSD': 150  # not here???
}

# Add fee amount column 
merge_df['OP Boost'] = 0

# Populate fee amount by matching pair in pools
for index, row in merge_df.iterrows():
    pair = row['Pair']
    if pair in pools:
        merge_df.at[index, 'OP Boost'] = pools[pair]

SyntaxError: invalid syntax (3518228025.py, line 7)

In [ ]:
# Export the final_df DataFrame to a CSV file
merge_df.to_csv('final_data.csv', index=False)